In [9]:
from nltk.corpus import stopwords
import numpy as np 
import string

In [2]:
#import data
with open('Pride and Prejudice.txt', encoding = 'utf-8') as myfile:
    collection = myfile.read().split("\n\n") #\n\n denotes the double linebreak used for paragraph separation
    
for i in range(len(collection)):
    #replace "\n" in list items with a space
    collection[i] = collection[i].replace('\n',' ')
    #collection[i] = collection[i].replace("\'",'')
    
collection = [i.strip() for i in collection if len(i) > 1]

print(f"Number of paragraphs: {len(collection)}\n")
print(f"First paragraph:\n {collection[0]}\n")
print(f"Last paragraph:\n {collection[-1]}")

Number of paragraphs: 2190

First paragraph:
 ﻿The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen

Last paragraph:
 This Web site includes information about Project Gutenberg-tm, including how to make donations to the Project Gutenberg Literary Archive Foundation, how to help produce our new eBooks, and how to subscribe to our email newsletter to hear about new eBooks.


In [10]:
corpus = open('Pride and Prejudice.txt', encoding = 'utf-8').readlines()

In [11]:
def clean_word(text): 
    for word in stopwords.words('english'): 
        text = text.replace(' '+word+' ',' ')
    puncs = ['-', ':', '.', ';', ',', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '(', ')', '?', '[', ']', '!', '/', "'"]
    for _ in puncs:
        text = text.replace(_,'')
    return text

def make_paragraphs(text):
    paragraphs = []
    dumb = ''
    for _ in range(len(text)): 
        if text[_] == '\n':
            paragraphs.append(dumb)
            dumb = ''
            continue
        else: 
            dumb += text[_]
            
    for i in range(len(paragraphs)):
        paragraphs[i] = paragraphs[i].replace('\n', ' ').lower()
        paragraphs[i] = clean_word(paragraphs[i])
    
    return paragraphs

paragraphs = make_paragraphs(corpus)
print(len(paragraphs))
new_paragraphs = []
for _ in range(len(paragraphs)): 
    if len(paragraphs[_]) > 30:
        new_paragraphs.append(paragraphs[_])
print(len(new_paragraphs))

2394
1960


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=100,
                                stop_words='english')
vectors = vectorizer.fit_transform(new_paragraphs)

In [13]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=15, max_iter=10,
                                learning_method='batch',
                                random_state=25)
lda.fit(vectors)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=15, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=25, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [14]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


names = vectorizer.get_feature_names()
print_top_words(lda, names, 10)

Topic #0: project work received way away place long hope come wish
Topic #1: bingley mr sisters jane sister netherfield friend elizabeth till attention
Topic #2: cried oh lydia wickham certainly heard father dear left know
Topic #3: said hope subject sister feelings replied soon elizabeth away felt
Topic #4: lady catherine wickham mr young collins quite manner daughter make
Topic #5: miss bennet bingley house thought darcy elizabeth said seen soon
Topic #6: make happy love sure quite good dear daughter hear said
Topic #7: elizabeth saw said jane room looked morning came evening pleasure
Topic #8: say long come opinion jane mother said happiness shall sister
Topic #9: think wish like great believe world said really know place
Topic #10: darcy mr man elizabeth manner said time young wish colonel
Topic #11: know lizzy shall tell dear colonel think added aunt told
Topic #12: mr collins sir elizabeth charlotte lucas friend room house mrs
Topic #13: letter little family longbourn marriage so

In [15]:
import numpy as np
from collections import Counter

stick_breaks = np.asarray([])


def sample_stick_breaking_process(stick_breaks):
    s = np.random.rand()
    lst = np.where(s < stick_breaks)[0]
    if lst.size:
        return (stick_breaks, lst[0])
    else:
        # We have sampled a new category!
        # Break the stick in a new place and update the array!
        if stick_breaks.size == 0:
            start = 0
        else:
            start = stick_breaks[-1]
        width = 1.0 - start
        pos = start + width * np.random.rand()
        stick_breaks = np.append(stick_breaks, [pos])
        return (stick_breaks, len(stick_breaks))


N = 10000
samples = np.zeros(N)
for n in range(N):
    (stick_breaks, samples[n]) = sample_stick_breaking_process(stick_breaks)

print(stick_breaks)
print(Counter(samples))

[0.33438773 0.82925212 0.86288963 0.90270716 0.96534238 0.99518203
 0.99959159 0.99959902 0.99971906]
Counter({1.0: 4930, 0.0: 3375, 4.0: 621, 3.0: 382, 2.0: 343, 5.0: 317, 6.0: 29, 7.0: 1, 8.0: 1, 9.0: 1})
